In [14]:
import json

from tqdm import tqdm
import pandas as pd

In [5]:
# inputs
data_dir = '../data/intermediary'
fn_search = f'{data_dir}/searches.csv.gz'
fn_products = f'{data_dir}/products_with_meta.csv.gz'
fn_queries = '../data/input/combined_queries_with_source.csv'
fn_amazon = f'{data_dir}/amazon_private_label.csv.gz'

fn_top_searches = f'{data_dir}/top_searches.csv.gz'
fn_generic_searches = f'{data_dir}/generic_searches.csv.gz'

# outputs

In [3]:
df_products = pd.read_csv(fn_products, compression='gzip')
df_search = pd.read_csv(fn_search, compression='gzip')
df_amazon = pd.read_csv(fn_amazon, compression='gzip')
keywords = pd.read_csv(fn_queries)

In [6]:
df_top = pd.read_csv(fn_top_searches, compression='gzip')
df_generic = pd.read_csv(fn_generic_searches, compression='gzip')

In [36]:
df_top

,asin,product_name,stars,reviews,brand,product_url,is_prime,is_fresh,is_sponsored,is_featured_brand,...,sold_by,has_third_party_sellers,product_by_amazon,our_brands_carousel,ads,suggestions,n_ads,is_amazon,is_sold_by_amazon,is_shipped_by_amazon
0,B086VR2KY8,All-New Toshiba 50LF621U21 50-inch Smart 4K UH...,4.6,5282.0,NaN,/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,True,False,False,True,...,Best Buy,False,False,False,[],"['B086VR2KY8', 'B086VR2KY8', 'B086VR2KY8', 'B0...",0.0,True,False,False
1,B086VRR9K9,All-New Insignia NS-50DF710NA21 50-inch Smart ...,4.6,8569.0,NaN,/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,True,False,False,True,...,NaN,False,False,False,[],"['B086VRR9K9', 'B086VRR9K9', 'B086VRR9K9', 'B0...",0.0,True,False,False
2,B08DHFX4FV,"TCL 4K Smart LED TV, 50"" (50S435)",4.6,1314.0,NaN,/TCL-4K-Smart-LED-50S435/dp/B08DHFX4FV/ref=sr_...,True,False,False,False,...,Amazon.com,True,False,False,"[b'<div id=""ape_Detail_ams-detail-right-v2_des...","['B08DHFX4FV', 'B08DHFX4FV', 'B08DHFX4FV', 'B0...",4.0,False,True,True
3,B086VR2KY8,All-New Toshiba 50LF621U21 50-inch Smart 4K UH...,4.6,5282.0,NaN,/All-New-Toshiba-50LF621U21-50-inch-Vision/dp/...,True,False,False,False,...,Best Buy,False,False,False,[],"['B086VR2KY8', 'B086VR2KY8', 'B086VR2KY8', 'B0...",0.0,True,False,False
4,B086VRR9K9,All-New Insignia NS-50DF710NA21 50-inch Smart ...,4.6,8569.0,NaN,/All-New-Insignia-NS-50DF710NA21-50-inch-Smart...,True,False,False,False,...,NaN,False,False,False,[],"['B086VRR9K9', 'B086VRR9K9', 'B086VRR9K9', 'B0...",0.0,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187529,B075N8W9VK,"163 Piece STEM Toys Kit, Educational Construct...",4.7,7072.0,NaN,/Educational-Construction-Building-Blocks-Bric...,True,False,False,False,...,DynaMax,True,False,False,[],"['B075N8W9VK', 'B075N8W9VK', 'B075N8W9VK', 'B0...",0.0,False,False,True
187530,B07SJKW984,Flying Toy Ball Infrared Induction RC Flying T...,4.0,5445.0,NaN,/Infrared-Induction-Helicopter-Shining-Colorfu...,True,False,False,False,...,Fmaoun,True,False,False,"[b'<div id=""ape_Detail_ams-detail-right-v2_des...","['B07SJKW984', 'B07SJKW984', 'B07SJKW984', 'B0...",4.0,False,False,True
187531,B07TW2FYNV,"LLMoose Hover Soccer Ball Set of 2, Hover Ball...",4.4,3716.0,NaN,/Let-Loose-Moose-Hover-Soccer/dp/B07TW2FYNV/re...,True,False,False,False,...,KangoKids,True,False,False,"[b'<div id=""ape_Detail_ams-detail-right-v2_des...","['B07TW2FYNV', 'B07TW2FYNV', 'B07TW2FYNV', 'B0...",5.0,False,False,True
187532,B08CZYGBDT,MESIXI Bean Bag Toss Game Toy for Toddlers 2 3...,4.7,14.0,NaN,/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,True,False,True,False,...,Dobook,True,False,True,"[b'<div id=""ape_Detail_ams-detail-right-v2_des...","['B08CZYGBDT', 'B08CZYGBDT', 'B08CZYGBDT', 'B0...",4.0,False,False,True


In [4]:
amazon_asin = set(df_amazon.asin.tolist())
len(amazon_asin)

137420

In [7]:
df_products.iloc[0]

fn                         ../data/input/selenium-products/5/5D/B085DNV2T...
shipped_by                                                               NaN
sold_by                                                                  NaN
has_third_party_sellers                                                False
product_by_amazon                                                      False
our_brands_carousel                                                    False
ads                                                                       []
suggestions                                                               []
n_ads                                                                      0
asin                                                              B085DNV2T9
is_amazon                                                              False
is_sold_by_amazon                                                      False
is_shipped_by_amazon                                                   False

In [13]:
def jsonify(x):
    try:
        return json.loads(x.replace("'", '"'))
    except:
        return None
    
def is_overlap(value):
    """value is a list of asins"""
    if value['our_brands_carousel']:
        return True
    return not set(value['suggestions_']).isdisjoint(amazon_asin)

In [10]:
df_products = df_products[df_products.asin.isin(df_top.asin)]

In [11]:
df_products['suggestions_'] = df_products.suggestions.apply(jsonify)

In [15]:
tqdm.pandas()
df_products['suggestions_contain_amazon'] = df_products.progress_apply(is_overlap, axis=1)

100%|██████████| 124872/124872 [00:04<00:00, 30520.10it/s]


In [16]:
# perc of prods with Amazon
df_products[~df_products.suggestions_.apply(lambda x: len(x) == 0)]['suggestions_contain_amazon'].value_counts() / \
len(df_products[~df_products.suggestions_.apply(lambda x: len(x) == 0)])

False    0.674831
True     0.325169
Name: suggestions_contain_amazon, dtype: float64

In [17]:
# perc of prods with Amazon
df_products['suggestions_contain_amazon'].value_counts() / \
len(df_products)

False    0.684149
True     0.315851
Name: suggestions_contain_amazon, dtype: float64

In [21]:
# perc of prods with Amazon
# percent of non-Amazon products with Amazon products
df_products[
    (~df_products.asin.isin(amazon_asin)) & 
    (df_products.suggestions_.apply(lambda x: len(x) != 0))
]['suggestions_contain_amazon'].value_counts(normalize=True)

False    0.705293
True     0.294707
Name: suggestions_contain_amazon, dtype: float64

In [25]:
# number of product suggeestions
df_products.suggestions_.apply(lambda x: len(x)).mean()

33.69251713754885

In [26]:
# number of product suggeestions
df_products[~df_products.asin.isin(amazon_asin)].suggestions_.apply(lambda x: len(x)).mean()

33.91899887381709

In [27]:
# number of product suggeestions
df_products[df_products.asin.isin(amazon_asin)].suggestions_.apply(lambda x: len(x)).mean()

29.114169215086648

In [35]:
# how many products have zero ads
len(df_products[df_products.n_ads == 0]) / len(df_products)

0.5534707540521494

In [32]:
# avg number of ads
df_products[df_products.n_ads != 0].n_ads.mean()

4.119263257949389